# --- Day 8: Seven Segment Search ---
## Part 1
You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit seven-segment displays in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named a through g:
```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
```
So, to render a 1, only segments c and f would be turned on; the rest would be off. To render a 7, only segments a, c, and f would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires a through g, but those wires are connected to segments randomly. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits within a display use the same connections, though.)

So, you might know that only signal wires b and g are turned on, but that doesn't mean segments b and g are turned on: the only digit that uses two segments is 1, so it must mean segments c and f are meant to be on. With just that information, you still can't tell which wire (b/g) goes to which segment (c/f). For that, you'll need to collect more information.

For each display, you watch the changing signals for a while, make a note of all ten unique signal patterns you see, and then write down a single four digit output value (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

For example, here is what you might see in a single entry in your notes:

acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf

(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because 7 is the only digit that uses three segments, dab in the above example means that to render a 7, signal lines d, a, and b are on. Because 4 is the only digit that uses four segments, eafb means that to render a 4, signal lines e, a, f, and b are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (cdfeb fcadb cdfeb cdbaf) use five segments and are more difficult to deduce.

For now, focus on the easy digits. Consider this larger example:
```
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce
```
Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?


In [1]:
import numpy as np

In [2]:
output_digit_segment_count = []
with open("../data/day08.dat", "r") as IH:
    for line in IH:
        output_digit_segment_count.extend([len(x) for x in line.strip().split(" | ")[1].split(" ")])
output_digit_segment_count = np.array(output_digit_segment_count)
sum(output_digit_segment_count == 2) + sum(output_digit_segment_count == 4) + sum(output_digit_segment_count == 3) + sum(output_digit_segment_count == 7)

548

### Decode

In [3]:
from typing import Dict, List

In [4]:
class FourDigitScreen:
    unique_signals: List[str]
    digit_to_signal: Dict[int, str]
    signal_to_digit: Dict[str, int]
    output_signal: List[str]
    
    def __init__(self, note: str):
        self.unique_signals, self.output_signal = [x.split(" ") for x in note.strip().split(" | ")]
        self.unique_signals = ["".join(sorted(x)) for x in self.unique_signals]
        self.output_signal = ["".join(sorted(x)) for x in self.output_signal]
        self.digit_to_signal = {}
        self.signal_to_digit = {}
        self.__decode()
    
    @property
    def output(self) -> int:
        return int("".join([str(self.signal_to_digit[word]) for word in self.output_signal]))
    
    def __decode(self) -> None:
        self.__decode_simple()
        self.__decode_3()
        self.__decode_9()
        self.__decode_6()
        self.__decode_5()
        self.__decode_rest()
    
    def __decode_simple(self) -> None:
        # Identify obvious digits
        for signal in self.unique_signals:
            if len(signal) == 2:
                self.digit_to_signal[1] = signal
                self.signal_to_digit[signal] = 1
            elif len(signal) == 3:
                self.digit_to_signal[7] = signal
                self.signal_to_digit[signal] = 7
            elif len(signal) == 4:
                self.digit_to_signal[4] = signal
                self.signal_to_digit[signal] = 4
            elif len(signal) == 7:
                self.digit_to_signal[8] = signal
                self.signal_to_digit[signal] = 8
                
    def __decode_3(self) -> None:
        for signal in self.unique_signals:
            if len(signal) == 5 and all(c in signal for c in self.digit_to_signal[7]):
                self.digit_to_signal[3] = signal
                self.signal_to_digit[signal] = 3

    def __decode_9(self) -> None:
        for signal in self.unique_signals:
            if len(signal) == 6 and all(c in signal for c in self.digit_to_signal[4]):
                self.digit_to_signal[9] = signal
                self.signal_to_digit[signal] = 9

    def __decode_6(self) -> None:
        for signal in self.unique_signals:
            if len(signal) == 6 and sum(c in signal for c in self.digit_to_signal[1]) == 1:
                self.digit_to_signal[6] = signal
                self.signal_to_digit[signal] = 6
                
    def __decode_5(self) -> None:
        for signal in self.unique_signals:
            if len(signal) == 5 and all(c in self.digit_to_signal[6] for c in signal):
                self.digit_to_signal[5] = signal
                self.signal_to_digit[signal] = 5

    def __decode_rest(self) -> None:
        if len(self.digit_to_signal.keys()) != 8:
            return
        for signal in self.unique_signals:
            if len(signal) == 5 and signal not in self.signal_to_digit:
                self.digit_to_signal[2] = signal
                self.signal_to_digit[signal] = 2
            if len(signal) == 6 and signal not in self.signal_to_digit:
                self.digit_to_signal[0] = signal
                self.signal_to_digit[signal] = 0

    def __repr__(self):
        return f"Signals: {self.unique_signals}\nOutput: {self.output_signal}\nS2D: {self.signal_to_digit}\nD2S: {self.digit_to_signal}"

with open("../data/day08.example.dat", "r") as IH:
    for line in IH:
        FDS = FourDigitScreen(line)
        print(f"{' '.join(FDS.output_signal)}: {FDS.output}")

abcdefg bcdef bcdefg bceg: 8394
bcdefg bcg abcdefg cg: 9781
cg cg abcdfg bcg: 1197
abcdef abcde acdefg bc: 9361
cefg abcdefg bfg abefg: 4873
abcdefg abce ac abcdefg: 8418
cefg bcdef cefg abcdefg: 4548
de abcefg abcdg bcefg: 1625
abcdefg bcg cg bcg: 8717
aefg abcfg fg abceg: 4315


## --- Part Two ---

Through a little deduction, you should now be able to determine the remaining digits. Consider again the first example above:

acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf

After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:
```
 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc
```
So, the unique signal patterns would correspond to the following digits:
```
    acedgfb: 8
    cdfbe: 5
    gcdfa: 2
    fbcad: 3
    dab: 7
    cefabd: 9
    cdfgeb: 6
    eafb: 4
    cagedb: 0
    ab: 1
```
Then, the four digits of the output value can be decoded:
```
    cdfeb: 5
    fcadb: 3
    cdfeb: 5
    cdbaf: 3
```
Therefore, the output value for this entry is 5353.

Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:
```
    fdgacbe cefdb cefbgd gcbe: 8394
    fcgedb cgb dgebacf gc: 9781
    cg cg fdcagb cbg: 1197
    efabcd cedba gadfec cb: 9361
    gecf egdcabf bgf bfgea: 4873
    gebdcfa ecba ca fadegcb: 8418
    cefg dcbef fcge gbcadfe: 4548
    ed bcgafe cdgba cbgef: 1625
    gbdfcae bgc cg cgb: 8717
    fgae cfgab fg bagce: 4315
```
Adding all of the output values in this larger example produces 61229.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?


In [5]:
output_values = []
with open("../data/day08.dat", "r") as IH:
    for line in IH:
        FDS = FourDigitScreen(line)
        if not 10 == len(FDS.signal_to_digit):
            print(FDS)
        output_values.append(FDS.output)
sum(output_values)

1074888